# Scraping of questions from dotazy.ujc.cas.cz

In [2]:
import requests
from bs4 import BeautifulSoup

In [22]:
for page_num in range(1, 2):
    page_url = f"https://dotazy.ujc.cas.cz/odpovedi/?page={page_num}&items=2"
    response = requests.get(page_url)
    soup = BeautifulSoup(response.text, 'html.parser')

soup

<!DOCTYPE html>

<html lang="cs">
<head>
<!-- Required meta tags -->
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1, shrink-to-fit=no" name="viewport"/>
<meta content="Jazyková poradna Ústavu pro jazyk český AV ČR na webu zveřejňuje odpovědi na jazykovou problematiku. Návštěvníci webu mohou odpovědi vyhledávat dle dotazu, klíčového slova nebo odpovědi." name="description"/>
<meta content="Jazyková poradna, ÚJČ, ÚJČ AV ČR, dotazy, odpovědi, vyhledávání, vyhledávání odpovědí, jazyková problematika, jazykové poradenství, český jazyk, pravopis, gramatika" name="keyword"/>
<meta content="Západočeská univerzita v Plzni; Ústav pro jazyk český AV ČR" name="author"/>
<link href="/_images/_frontend/favicon/dotazy-ikona-v1.png" rel="icon" type="image/png"/>
<!-- Libraries -->
<link href="/_web_bower_components/_frontend/node_modules/bootstrap/dist/css/bootstrap.min.css?time=2024-05-08" rel="stylesheet"/>
<link href

In [64]:
searched_fields = ['Dotaz', 'Konkrétní dotaz', 'Klíčové slovo', 'Odpověď', 'Poslední užití']

#map searched fields to english alternatives
searched_fields_map = {
    'Dotaz': 'question',
    'Konkrétní dotaz': 'specific_question',
    'Klíčové slovo': 'keyword',
    'Odpověď': 'answer',
    'Poslední užití': 'last_usage'
}

In [70]:
def extract_results_from_page(soup: BeautifulSoup, searched_fields: list, searched_fields_map: dict):
    results = []

    main_divs = soup.find_all('div', class_='row card-body')
    if main_divs:
        for main_div in main_divs:
            # Check if the div contains the searched fields
            result = {}
            for field in searched_fields:
                field_text = main_div.find(string=lambda t: t and field in t)
                if field_text:
                    extracted_field = field_text.find_next('span').text.strip()
                    result[searched_fields_map[field]] = extracted_field

            #extract Zvažované varianty
            variants_div = main_div.find('div', string='Zvažované varianty:')
            if variants_div:
                variants = variants_div.find_next('div').find_all('span', class_='bold')
                result['variants'] = [var.get_text(strip=True) for var in variants]

            #if results is not empty dictionary
            if result:
                results.append(result)

    return results


In [71]:
extract_results_from_page(soup, searched_fields=searched_fields, searched_fields_map=searched_fields_map)

[{'question': 'Zápis přejatého slova',
  'specific_question': 'Předpokládám správně, že zjednodušování th na t neplatí pro slovo théta? Nebo se píše i téta?',
  'keyword': 'théta',
  'answer': 'Váš předpoklad je správný. Náležitá podoba pojmenování písmena řecké abecedy je v souladu s jazykovými příručkami jedině théta. Výslovnost může být [théta] i [téta] (viz např. IJP).',
  'last_usage': '25.10.2023',
  'variants': ['théta', 'téta']},
 {'question': 'Zápis přejatého slova',
  'specific_question': 'Jde mi o skupinu th v přejatých slovech užívaných v chemickém názvosloví. Konkrétně o psaní slova bromthymolová modř.',
  'keyword': 'bronthymolový',
  'answer': 'Souhlásková skupina th se dnes ve zdomácnělých přejatých slovech obvykle píše jako t. Např.: antologie, estetika, etan, katedra, mytologie, patos, syntéza. Podle obecných pravopisných zásad (uvedených ve výkladové části PČP nebo v IJP v kapitole Počešťování přejatých slov včetně vlastních jmen) však lze i slova, která se běžně píš

In [72]:
from tqdm import tqdm

def extract_all_results():
    all_results = []
    for page_num in tqdm(range(1, 26)):
        page_url = f"https://dotazy.ujc.cas.cz/odpovedi/?page={page_num}&items=500"
        response = requests.get(page_url)
        soup = BeautifulSoup(response.text, 'html.parser')
        all_results.extend(extract_results_from_page(soup, searched_fields=searched_fields, searched_fields_map=searched_fields_map))

    return all_results


In [73]:
all_results = extract_all_results()

100%|██████████| 25/25 [01:32<00:00,  3.72s/it]


In [74]:
all_results

[{'question': 'Zápis přejatého slova',
  'specific_question': 'Předpokládám správně, že zjednodušování th na t neplatí pro slovo théta? Nebo se píše i téta?',
  'keyword': 'théta',
  'answer': 'Váš předpoklad je správný. Náležitá podoba pojmenování písmena řecké abecedy je v souladu s jazykovými příručkami jedině théta. Výslovnost může být [théta] i [téta] (viz např. IJP).',
  'last_usage': '25.10.2023',
  'variants': ['théta', 'téta']},
 {'question': 'Zápis přejatého slova',
  'specific_question': 'Jde mi o skupinu th v přejatých slovech užívaných v chemickém názvosloví. Konkrétně o psaní slova bromthymolová modř.',
  'keyword': 'bronthymolový',
  'answer': 'Souhlásková skupina th se dnes ve zdomácnělých přejatých slovech obvykle píše jako t. Např.: antologie, estetika, etan, katedra, mytologie, patos, syntéza. Podle obecných pravopisných zásad (uvedených ve výkladové části PČP nebo v IJP v kapitole Počešťování přejatých slov včetně vlastních jmen) však lze i slova, která se běžně píš

In [75]:
import pandas as pd

df = pd.DataFrame(all_results)
df

,question,specific_question,keyword,answer,last_usage,variants
0,Zápis přejatého slova,"Předpokládám správně, že zjednodušování th na ...",théta,Váš předpoklad je správný. Náležitá podoba poj...,25.10.2023,"[théta, téta]"
1,Zápis přejatého slova,Jde mi o skupinu th v přejatých slovech užívan...,bronthymolový,Souhlásková skupina th se dnes ve zdomácnělých...,25.10.2023,"[bronthymolový, brontymolový]"
2,Zápis přejatého slova,Jde mi o skupinu th v přejatých slovech užívan...,etylen; ethylen,Souhlásková skupina th se dnes ve zdomácnělých...,25.10.2023,"[etylen, ehtylen]"
3,Zápis přejatého slova,"Jak v českém textu napsat filagrin, popř. prof...",filagrin; filaggrin,"Jde o poměrně úzce odborný lékařský termín, pr...",10.4.2024,"[filagrin, filaggrin]"
4,Tvar zájmena – ostatní,Jaký bude zájmena „všechen“ v následující větě...,všechen; několikanásobný větný člen; přívlastek,Pokud shodný přívlastek předchází dvěma souřad...,23.10.2024,NaN
...,...,...,...,...,...,...
12358,"Nejasná životnost, kolísání","Je smajlík životné podstatné jméno, nebo neživ...",smajlík,Podstatné jméno smajlík sice kodifikační příru...,31.1.2017,NaN
12359,Úprava odstavců,Potýkáme se s členěním dopisů pro zákazníky na...,odstavec,"Hranice, na jakých místech by měl být text čle...",20.3.2015,NaN
12360,Výraz na konci řádku,Když se mi na konci řádku objeví spojovník naz...,spojovník,Pokud se spojovník objeví na konci řádku a nen...,23.5.2019,NaN
12361,Oslovování,Je možné při oslovování ponechat příjmení v mu...,1. pád; 5. pád,V psané i mluvené komunikaci doporučujeme užív...,2.9.2021,"[pane Novák, pane Nováku]"


In [76]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12363 entries, 0 to 12362
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   question           12363 non-null  object
 1   specific_question  11287 non-null  object
 2   keyword            11505 non-null  object
 3   answer             12363 non-null  object
 4   last_usage         12175 non-null  object
 5   variants           4967 non-null   object
dtypes: object(6)
memory usage: 579.6+ KB


In [78]:
df["last_usage"] = pd.to_datetime(df["last_usage"], format='%d.%m.%Y')
df

,question,specific_question,keyword,answer,last_usage,variants
0,Zápis přejatého slova,"Předpokládám správně, že zjednodušování th na ...",théta,Váš předpoklad je správný. Náležitá podoba poj...,2023-10-25,"[théta, téta]"
1,Zápis přejatého slova,Jde mi o skupinu th v přejatých slovech užívan...,bronthymolový,Souhlásková skupina th se dnes ve zdomácnělých...,2023-10-25,"[bronthymolový, brontymolový]"
2,Zápis přejatého slova,Jde mi o skupinu th v přejatých slovech užívan...,etylen; ethylen,Souhlásková skupina th se dnes ve zdomácnělých...,2023-10-25,"[etylen, ehtylen]"
3,Zápis přejatého slova,"Jak v českém textu napsat filagrin, popř. prof...",filagrin; filaggrin,"Jde o poměrně úzce odborný lékařský termín, pr...",2024-04-10,"[filagrin, filaggrin]"
4,Tvar zájmena – ostatní,Jaký bude zájmena „všechen“ v následující větě...,všechen; několikanásobný větný člen; přívlastek,Pokud shodný přívlastek předchází dvěma souřad...,2024-10-23,NaN
...,...,...,...,...,...,...
12358,"Nejasná životnost, kolísání","Je smajlík životné podstatné jméno, nebo neživ...",smajlík,Podstatné jméno smajlík sice kodifikační příru...,2017-01-31,NaN
12359,Úprava odstavců,Potýkáme se s členěním dopisů pro zákazníky na...,odstavec,"Hranice, na jakých místech by měl být text čle...",2015-03-20,NaN
12360,Výraz na konci řádku,Když se mi na konci řádku objeví spojovník naz...,spojovník,Pokud se spojovník objeví na konci řádku a nen...,2019-05-23,NaN
12361,Oslovování,Je možné při oslovování ponechat příjmení v mu...,1. pád; 5. pád,V psané i mluvené komunikaci doporučujeme užív...,2021-09-02,"[pane Novák, pane Nováku]"


In [79]:
df.describe()

,last_usage
count,12175
mean,2019-03-19 19:23:21.264887296
min,2015-01-01 00:00:00
25%,2017-08-16 00:00:00
50%,2018-10-11 00:00:00
75%,2020-10-26 00:00:00
max,2024-10-23 00:00:00


In [81]:
import ftfy
#fix text
df["answer"] = df["answer"].apply(lambda x: ftfy.fix_text(x).strip() if type(x) == str else x)
df["specific_question"] = df["specific_question"].apply(lambda x: ftfy.fix_text(x).strip()if type(x) == str else x)
df["question"] = df["question"].apply(lambda x: ftfy.fix_text(x).strip()if type(x) == str else x)
df["keyword"] = df["keyword"].apply(lambda x: ftfy.fix_text(x).strip()if type(x) == str else x)
df

,question,specific_question,keyword,answer,last_usage,variants
0,Zápis přejatého slova,"Předpokládám správně, že zjednodušování th na ...",théta,Váš předpoklad je správný. Náležitá podoba poj...,2023-10-25,"[théta, téta]"
1,Zápis přejatého slova,Jde mi o skupinu th v přejatých slovech užívan...,bronthymolový,Souhlásková skupina th se dnes ve zdomácnělých...,2023-10-25,"[bronthymolový, brontymolový]"
2,Zápis přejatého slova,Jde mi o skupinu th v přejatých slovech užívan...,etylen; ethylen,Souhlásková skupina th se dnes ve zdomácnělých...,2023-10-25,"[etylen, ehtylen]"
3,Zápis přejatého slova,"Jak v českém textu napsat filagrin, popř. prof...",filagrin; filaggrin,"Jde o poměrně úzce odborný lékařský termín, pr...",2024-04-10,"[filagrin, filaggrin]"
4,Tvar zájmena – ostatní,"Jaký bude zájmena ""všechen"" v následující větě...",všechen; několikanásobný větný člen; přívlastek,Pokud shodný přívlastek předchází dvěma souřad...,2024-10-23,NaN
...,...,...,...,...,...,...
12358,"Nejasná životnost, kolísání","Je smajlík životné podstatné jméno, nebo neživ...",smajlík,Podstatné jméno smajlík sice kodifikační příru...,2017-01-31,NaN
12359,Úprava odstavců,Potýkáme se s členěním dopisů pro zákazníky na...,odstavec,"Hranice, na jakých místech by měl být text čle...",2015-03-20,NaN
12360,Výraz na konci řádku,Když se mi na konci řádku objeví spojovník naz...,spojovník,Pokud se spojovník objeví na konci řádku a nen...,2019-05-23,NaN
12361,Oslovování,Je možné při oslovování ponechat příjmení v mu...,1. pád; 5. pád,V psané i mluvené komunikaci doporučujeme užív...,2021-09-02,"[pane Novák, pane Nováku]"


In [82]:
df["id"] = df.index
df

,question,specific_question,keyword,answer,last_usage,variants,id
0,Zápis přejatého slova,"Předpokládám správně, že zjednodušování th na ...",théta,Váš předpoklad je správný. Náležitá podoba poj...,2023-10-25,"[théta, téta]",0
1,Zápis přejatého slova,Jde mi o skupinu th v přejatých slovech užívan...,bronthymolový,Souhlásková skupina th se dnes ve zdomácnělých...,2023-10-25,"[bronthymolový, brontymolový]",1
2,Zápis přejatého slova,Jde mi o skupinu th v přejatých slovech užívan...,etylen; ethylen,Souhlásková skupina th se dnes ve zdomácnělých...,2023-10-25,"[etylen, ehtylen]",2
3,Zápis přejatého slova,"Jak v českém textu napsat filagrin, popř. prof...",filagrin; filaggrin,"Jde o poměrně úzce odborný lékařský termín, pr...",2024-04-10,"[filagrin, filaggrin]",3
4,Tvar zájmena – ostatní,"Jaký bude zájmena ""všechen"" v následující větě...",všechen; několikanásobný větný člen; přívlastek,Pokud shodný přívlastek předchází dvěma souřad...,2024-10-23,NaN,4
...,...,...,...,...,...,...,...
12358,"Nejasná životnost, kolísání","Je smajlík životné podstatné jméno, nebo neživ...",smajlík,Podstatné jméno smajlík sice kodifikační příru...,2017-01-31,NaN,12358
12359,Úprava odstavců,Potýkáme se s členěním dopisů pro zákazníky na...,odstavec,"Hranice, na jakých místech by měl být text čle...",2015-03-20,NaN,12359
12360,Výraz na konci řádku,Když se mi na konci řádku objeví spojovník naz...,spojovník,Pokud se spojovník objeví na konci řádku a nen...,2019-05-23,NaN,12360
12361,Oslovování,Je možné při oslovování ponechat příjmení v mu...,1. pád; 5. pád,V psané i mluvené komunikaci doporučujeme užív...,2021-09-02,"[pane Novák, pane Nováku]",12361


In [83]:
df.to_json('../../data/it_datasets/dotazy_ujc_cas.jsonl', orient='records', lines=True)

In [84]:
from datasets import Dataset

hf_dataset = Dataset.from_pandas(df, preserve_index=False)
hf_dataset

Dataset({
    features: ['question', 'specific_question', 'keyword', 'answer', 'last_usage', 'variants', 'id'],
    num_rows: 12363
})

In [85]:
hf_dataset.save_to_disk("../../data/it_datasets/questions_ujc_cas")

Saving the dataset (0/1 shards):   0%|          | 0/12363 [00:00<?, ? examples/s]

In [86]:
hf_dataset.push_to_hub("ctu-aic/questions_ujc_cas_cs", private=True, token="")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/13 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/ctu-aic/questions_ujc_cas_cs/commit/7ba2d25bf461dda36242783260713bb1b30dc036', commit_message='Upload dataset', commit_description='', oid='7ba2d25bf461dda36242783260713bb1b30dc036', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/ctu-aic/questions_ujc_cas_cs', endpoint='https://huggingface.co', repo_type='dataset', repo_id='ctu-aic/questions_ujc_cas_cs'), pr_revision=None, pr_num=None)